# **Instruções Gerais**

- **Copie** este notebook, junto com as imagens da pasta para o seu Google Drive da UFV
- Resolva o que está sendo pedido no próprio notebook
- Lembre-se de **montar o seu Google Drive** no sistema de arquivos do notebook (ícone da pastinha, na barra de ferramentas à esquerda)
- Entregue, como resposta à atividade no **Moodle**

- **Obs.:**
  - Lembre-se de **salvar** o notebook de tempos em tempos
  - Envie **apenas** o notebook. Não há necessidade de enviar as imagens

# **1. Preâmbulo**

## **1.1. Identifique-se**

Complete o que está sendo pedido na célula a seguir

In [ ]:
# Informe, dentro das strings, seu nome e número de matrícula
estudante = 'Erick Lima Figueiredo'
matricula = '98898'

## **1.2. Importações de módulos Python**

- As bibliotecas necessárias para a atividade já se encontram importadas na célula a seguir. Mas você pode adicionar novas importações, se achar necessário

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

- Altere o caminho abaixo para a pasta onde vc copiou os arquivos

In [ ]:
os.chdir('/content/drive/MyDrive/Praticas PDI')

## **1.3. Funções auxiliares já implementadas**

In [ ]:
def gera_imagem(arr):
  arrImg = arr.round().clip(0,255).astype(np.uint8)
  return Image.fromarray(arrImg)

In [ ]:
def exibe_mosaico(imagens, legendas, colunas=4, arquivo=None):
  # Largura e altura básicos, de referência, das imagens
  l = 8
  a = 6

  n_imagens = len(imagens)
  if len(imagens) < colunas:
    colunas = len(imagens)
  linhas = n_imagens // colunas + int(n_imagens % colunas > 0)

  fig = plt.figure(figsize=(l*colunas,a*linhas))
  ax = []

  i = j = 0
  for pos, img in enumerate(imagens):
    i = pos // colunas
    j = pos % colunas
    ax.append(fig.add_subplot(linhas, colunas, pos+1))
    ax[-1].axis('off')
    if img.mode == 'L' or img.mode == '1':
      img = img.convert('RGB')
    arr = np.asarray(img)
    ax[-1].imshow(arr)
    ax[-1].set_title(legendas[pos])

  fig.tight_layout()

  if arquivo is not None:
    plt.savefig(arquivo, dpi=300)

  plt.show()

In [ ]:
def rmse(arr1, arr2):
  return np.sqrt(np.sum((arr1 - arr2)**2))

In [ ]:
def corrige(funcao, sufixo, pars=None):
  imagens = [
      'praia_floripa.png',
      'papagaio.png',
      'aguia.png',
      'camboriu.png',
      'pimentas.png'
  ]
  acertos = 0
  for arq in imagens:
    print('Testando arquivo %s...' % arq, end='')
    img = Image.open(arq)
    if pars is None:
      imgResult = funcao(img)
    else:
      imgResult = funcao(img, **pars)

    arrResult = np.asarray(imgResult).astype(int)
    arqGab = arq[:arq.rfind('.')] + '-' + sufixo + '.npy'
    arrGab = np.load(arqGab, allow_pickle=True)

    if rmse(arrResult, arrGab) == 0.0:
      print(' Ok!')
      acertos += 1
    else:
      print(' Não passou!')

  perc = acertos / len(imagens) * 100
  print('Você acertou %d de %d resultados (%.2f%%)' % (acertos, len(imagens), perc))

# **2. Dica**

Um objeto da classe `PIL Image` possui vários métodos úteis, como `resize`, para redimensionar uma imagem, `copy`, para copiar, `crop`, para fazer um corte ou `convert`, para converter uma imagem para diferentes modelos de representação de cores.

A função `convert` admite alguns parâmetros, como uma matriz de conversão, uma paleta de cores, dentre outros. Vamos focar aqui no parâmetro `mode`, cujos principais valores são:

- `'1'`: imagem binária
- `'L'`: imagem monocromática, em tons de cinza (usando a *luminosity*)
- `'RGB'`: imagem RGB
- `'RGBA'`: imagem RGB com canal de transparência (*alpha*)

Para saber mais sobre os possíveis modos, consulte [este link](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes).

Execute a célula abaixo, para um exemplo de uso do método `convert`:

In [ ]:
img = Image.open('camboriu.png')
imgCinza = img.convert('L')

imgP = Image.open('pimentas.png').convert('RGB')
imgBin = imgP.convert('1')

exibe_mosaico([img, imgCinza, imgP, imgBin], ['Original', 'Monocromática', 'Original', 'Binária'])

O parâmetro `mode` é mapeado para um atributo de mesmo nome, no objeto `PIL Image`. Execute o exemplo abaixo:

In [ ]:
img = Image.open('pimentas.png')
print('Modo da imagem:', img.mode)

In [ ]:
np.asarray(img.copy().convert('L')).shape

# **3. Exercícios**

## **3.1. *Dithering* monocromático**

Programe o corpo da função `atkinson_dith`, abaixo, de forma que a mesma implemente o *dithering* de Atkinson, cujo algoritmo se encontra no slide 77 do material de aula.

Parâmetro:

- `img`: Imagem de entrada. Se for imagem RGB, converta para tons de cinza como primeiro passo da função (use o método `convert`, explicado acima). Se já for uma imagem monocromática, salte este passo.

Retorno:

- Objeto `PIL Image`, em **tons de cinza**.

In [ ]:
def atkinson_dith(img):
  ######################## COMPLETE COM SEU CÓDIGO #####################
  img_copy = np.asarray(img.copy() if img.mode == 'L' else img.convert('L')).astype(float)

  h, w = img_copy.shape

  for i in range(h):
    for j in range(w):
      v = img_copy[i, j]

      img_copy[i, j] = 255 if v >= (255//2) else 0

      qtd_err = v-img_copy[i, j]

      if j+1 < w:
        img_copy[i, j+1] += qtd_err/8

      if j+2 < w:
        img_copy[i, j+2] += qtd_err/8

      if i+1 < h and j-1 >= 0:
        img_copy[i+1, j-1] += qtd_err/8

      if i+1 < h:
        img_copy[i+1, j] += qtd_err/8

      if i+1 < h and j+1 < w:
        img_copy[i+1, j+1] += qtd_err/8

      if i+2 < h:
        img_copy[i+2, j] += qtd_err/8

  return gera_imagem(img_copy)


#### **Teste sua solução**

Execute a célula abaixo.

In [ ]:
img = Image.open('praia_floripa.png')
imgDith = atkinson_dith(img)

imgs = [img, imgDith]
legs = ['Original', 'Atkinson Dithering']

img2 = Image.open('papagaio.png')
img2Dith = atkinson_dith(img2)

imgs += [img2, img2Dith]
legs += ['Original', 'Atkinson Dithering']

exibe_mosaico(imgs, legs)

#### **Auto correção**

Estime sua nota deste exercício, executando a célula abaixo

In [ ]:
corrige(atkinson_dith, 'dith')

## **3.2. *Dithering* colorido**

Programe o corpo da função `atkinson_cor`, abaixo, de forma que a mesma implemente o *dithering* de Atkinson, em uma versão colorida. Isto é, o efeito deve ser aplicado em cada um dos canais RGB, independentemente. Você pode utilizar sua implementação anterior na solução deste exercício.

Parâmetro:

- `img`: Imagem de entrada. Se a imagem não for RGB, converta a mesma para o padrão RGB como primeiro passo da função (use o método `convert`, explicado acima). Ou seja, se a entrada for monocromática, ela será convertida para RGB. O resultado será uma imagem com 3 canais, cujos valores são repetidos em todos canais.

Retorno:

- Objeto `PIL Image`, em **modo RGB**.

In [ ]:
def atkinson_cor(img):
  ######################## COMPLETE COM SEU CÓDIGO #####################
  img_copy = np.asarray(img.copy() if img.mode == 'RGB' else img.convert('RGB')).astype(float)

  h, w, _ = img_copy.shape

  for i in range(h):
    for j in range(w):
      v = img_copy[i, j].copy()

      img_copy[i, j, 0] = 255 if v[0] >= (255//2) else 0
      img_copy[i, j, 1] = 255 if v[1] >= (255//2) else 0
      img_copy[i, j, 2] = 255 if v[2] >= (255//2) else 0

      qtd_err = v - img_copy[i, j]

      if j+1 < w:
        img_copy[i, j+1] += qtd_err/8

      if j+2 < w:
        img_copy[i, j+2] += qtd_err/8

      if i+1 < h and j-1 >= 0:
        img_copy[i+1, j-1] += qtd_err/8

      if i+1 < h:
        img_copy[i+1, j] += qtd_err/8

      if i+1 < h and j+1 < w:
        img_copy[i+1, j+1] += qtd_err/8

      if i+2 < h:
        img_copy[i+2, j] += qtd_err/8

  return gera_imagem(img_copy)




#### **Teste sua solução**

Execute a célula abaixo.

In [ ]:
img = Image.open('pimentas.png')
imgDith = atkinson_cor(img)

imgs = [img, imgDith]
legs = ['Original', 'Atkinson Dithering']

img2 = Image.open('aguia.png')
img2Dith = atkinson_cor(img2)

imgs += [img2, img2Dith]
legs += ['Original', 'Atkinson Dithering']

exibe_mosaico(imgs, legs, colunas=2)

#### **Auto correção**

Estime sua nota deste exercício, executando a célula abaixo

In [ ]:
corrige(atkinson_cor, 'dithC')

## **3.3. Pixelização**

Programe o corpo da função `pixeliza`, abaixo, de forma que a mesma implemente o efeito de "superpixels", conforme mostrado nos slides 74 a 76 dos slides do Capítulo.

Parâmetros:

- `img`: Imagem de entrada. Se a imagem não for RGB, converta a mesma para o padrão RGB como primeiro passo da função (use o método `convert`, explicado acima). Ou seja, se a entrada for monocromática, ela será convertida para RGB. O resultado será uma imagem com 3 canais, cujos valores são repetidos em todos canais.

- `tam_bloco`: número inteiro. Indica o tamanho do "superpixel". Ou seja, o superpixel será composto pelo agrupamento de $tam\_bloco \times tam\_bloco$ pixels da imagem original. Caso as dimensões da imagem original não sejam múltiplos exatos de $tam\_bloco$, gere blocos menores nas últimas linhas/colunas da imagem.
  - Por exemplo, se uma imagem tem 23 colunas e queremos agrupar em blocos de tamanho 3, teremos $23 // 3 = 7$ agrupamentos de pixels com 3 colunas e um agrupamento com $23 \% 3 = 2$ colunas. Isto é, teremos $8$ superpixels na dimensão das colunas
  - Analogamente, faz-se o mesmo processo nas linhas

- `funcao`: a função que fará a agregação dos valores para produzir o valor final do superpixel. Por exemplo: `np.average`, `np.median`, `np.max` etc. A função informada tem que dar suporte ao parâmetro `axis`, que é o caso destas da `numpy` mencionadas.

Retorno:

- Objeto `PIL Image`, em **modo RGB**.

In [ ]:
def pixeliza(img, tam_bloco, funcao):
  ######################## COMPLETE COM SEU CÓDIGO #####################
  img_copy = np.asarray(img.copy() if img.mode == 'RGB' else img.convert('RGB')).astype(int)

  h = img_copy.shape[0] // tam_bloco + 1 if img_copy.shape[0] % tam_bloco != 0 else img_copy.shape[0]//tam_bloco
  w = img_copy.shape[1] // tam_bloco + 1 if img_copy.shape[1] % tam_bloco != 0 else img_copy.shape[1]//tam_bloco

  for i in range(h):
    for j in range(w):
      begin_h = tam_bloco * i
      end_h = tam_bloco * (i+1) if tam_bloco*(i+1) < h else h

      begin_w = tam_bloco * j
      end_w = tam_bloco * (j+1) if tam_bloco*(j+1) < w else w

      aux_r=[]
      aux_g=[]
      aux_b=[]

      for k in range(begin_h, end_h):
        for l in range(begin_w, end_w):
          aux_r.append(img_copy[k,l,0])
          aux_g.append(img_copy[k,l,1])
          aux_b.append(img_copy[k,l,2])

      for k in range(begin_h, end_h):
        for l in range(begin_w, end_w):
          img_copy[k,l] = [new_r, new_g, new_b]

  return gera_imagem(img_copy)


In [ ]:
np.mean([1,2,3,4])

#### **Teste sua solução**

Execute a célula abaixo.

In [ ]:
img = Image.open('papagaio.png')
imgs = [img]
legs = ['Original']

imgPix = pixeliza(img, tam_bloco=8, funcao=np.median)
imgs.append(imgPix)
legs.append('Função: Mediana, Tam. Bloco=8')

imgPix = pixeliza(img, tam_bloco=4, funcao=np.median)
imgs.append(imgPix)
legs.append('Função: Mediana, Tam. Bloco=4')

imgPix = pixeliza(img, tam_bloco=8, funcao=np.min)
imgs.append(imgPix)
legs.append('Função: Min, Tam. Bloco=8')

img = Image.open('praia_floripa.png')
imgs.append(img)
legs.append('Original')

imgPix = pixeliza(img, tam_bloco=8, funcao=np.average)
imgs.append(imgPix)
legs.append('Função: Média, Tam. Bloco=8')

imgPix = pixeliza(img, tam_bloco=12, funcao=np.average)
imgs.append(imgPix)
legs.append('Função: Média, Tam. Bloco=12')

imgPix = pixeliza(img, tam_bloco=12, funcao=np.median)
imgs.append(imgPix)
legs.append('Função: Mediana, Tam. Bloco=12')

exibe_mosaico(imgs, legs)

#### **Auto correção**

Estime sua nota deste exercício, executando a célula abaixo

In [ ]:
corrige(pixeliza, 'pixAvg4', pars={'tam_bloco': 4, 'funcao': np.average})
print()
corrige(pixeliza, 'pixAvg8', pars={'tam_bloco': 8, 'funcao': np.average})
print()
corrige(pixeliza, 'pixMed8', pars={'tam_bloco': 8, 'funcao': np.median})
print()
corrige(pixeliza, 'pixMin4', pars={'tam_bloco': 4, 'funcao': np.min})

#### **Bons estudos!**

- Lembre-se de enviar **apenas** o notebook
- **Limpe as saídas em tela** antes de enviar. Isso diminui significativamente o tamanho do arquivo final e não gera problemas de recusa de submissão no Moodle. Para isso:
  - Vá no menu *Editar*
  - Escolha a opção *Limpar todas as saídas*
  - Salve o notebook